### Extract multiple invoices with reports from Chrome River
**Author**: Aaron MacGillivary (aaron@junctionapps.ca)

**Provided as a demo without any warranty of any kind, use at own risk**

Asked by Amy Moffitt in the forums, quick Python script provided as demo to show how one might accomplish this in Python. Amy's question was how to get all in a batch, but I'm uncertain if there is a way to get all the id's from a batch via Chrome River's api's. However, parsing an export file is done easily enough.

This is a proof of concept, you'd want to automate this with naming the output file appropriately, potentially emailing the resulting pdf to someone or whatever suits your business.

For this to work as written, you'll need to create a file called `cr_creds.py` saved in the same location as this notebook. It must be of the format:

```python
api_user = 'youraccountusername'
api_pass = 'goodmixoflettersnumbers'
api_base_url = 'servername_supplied_by_cr.chromeriver.com'
```

### Configurable options:

In [ ]:
export_file_name = "CR-InvoiceReport-prod-YOURCOMPANYCODE-2021-03-01-HHMMSSNN.xml"
output_file_name = "all_invoices.pdf"
# set include_full_pdf_report to True if you want the verbose version of the invoice image with report
include_full_pdf_report = False 

#### Shouldn't need to change anything below here
The rest of the code should require no changes, however, if your export file is not xml, or does not use the xml node `<invoiceExport>/<InvoiceHeader>/<ReportID>` you may need to alter the `get_invoices_from_export` function


In [ ]:
import io
import xml.etree.ElementTree as ET
import requests
from PyPDF2 import PdfFileMerger, PdfFileReader
from PyPDF2.utils import PdfReadError

In [ ]:
# However you manage credentials, do that. 
# for our purposes in this demo they are in a file called 'cr_creds.py' located in the directory as this notebook. That file looks like:
"""
api_user = 'youraccountusername'
api_pass = 'goodmixoflettersnumbers'
api_base_url = 'servername_supplied_by_cr.chromeriver.com'
"""
import cr_creds
url = f"https://{cr_creds.api_base_url}/receipts/doit"
credentials = {'un': cr_creds.api_user,
               'pw': cr_creds.api_pass,}      

In [ ]:
def get_invoice_image(invoice_id, url, credentials, include_report=False, stream=True):
    """ 
    Reaches out to Chrome River's invoice image api, using the getInvoiceImages
    https://chromeriver.force.com/s/article/Image-Integration-API-for-INVOICE
    """
    method_parameters = {'method': 'getInvoiceImages',
                         'invoiceID':invoice_id,
                        }
    if include_report:
        method_parameters['getPDFReport'] = 'true'  # small case on purpose
    return requests.post(url, data={**credentials, **method_parameters}, stream=stream)    

In [ ]:
def get_invoices_from_export(export_file):
    """
    Returns a list of ReportID's from a Chrome River export file.
    This may need to be tweaked if the export file has been modified from
    the normal offering from Chrome River.
    """
    tree = ET.parse(export_file)
    root = tree.getroot()
    return [invoice.text for invoice in root.findall("InvoiceHeader/ReportID")]

In [ ]:
def get_and_merge_pdfs(url, credentials, list_of_invoices, filename_to_create, include_report=False):
    """
    Takes a list of invoices and merges them into a single file named filename_to_create.
    Returns the file name created, and a list of exceptions for any that could not be read.
    """
    mergedInvoices = PdfFileMerger()
    exceptions = list()
    for invoice in list_of_invoices:
        invoice_pdf = get_invoice_image(invoice, url, credentials, include_report, stream=False)
        
        if invoice_pdf.status_code == 200:
            try:
                mergedInvoices.append(io.BytesIO(invoice_pdf.content))
            except PdfReadError:
                exceptions.append(invoice)
                
    if list_of_invoices:
        mergedInvoices.write(filename_to_create)
        return filename_to_create, exceptions
    else:
        return None

In [ ]:
# if you want to specify invoices manually, use this instead       
# invoices = ['050001111111',
#             '050002222222'  # adding as many as needed in this same way
#            ]    

# otherwise # get the list of invoices from a Chrome River export:
invoices = get_invoices_from_export(export_file_name)

# then merge them together
get_and_merge_pdfs(url, credentials, invoices, output_file_name, include_report=include_full_pdf_report)

In [ ]:
# test a single invoice
DEBUGGING = False
if DEBUGGING:
    import shutil
    invoice_id = '050000000001'
    r = get_invoice_image(invoice_id, url, credentials, include_report=False, stream=True)

    if r.status_code == 200:    
        with open(f'invoice_sample2_{invoice_id}.pdf', 'wb') as invoice:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, invoice)